In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import nltk
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import cross_val_score
from sklearn.metrics.pairwise import euclidean_distances
from IPython.display import display
import pickle
from wordcloud import WordCloud, STOPWORDS

In [2]:
pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns",None)
pd.set_option('display.max_colwidth', None)

In [3]:
dfmcn = pd.read_csv('movies2018-2023.csv',encoding = "ISO-8859-1")

In [4]:
dfmcn.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1100 entries, 0 to 1099
Data columns (total 13 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   title       1100 non-null   object 
 1   img         1100 non-null   object 
 2   type        1100 non-null   object 
 3   netflix_id  1100 non-null   int64  
 4   synopsis    1100 non-null   object 
 5   rating      1100 non-null   float64
 6   year        1100 non-null   int64  
 7   runtime     1100 non-null   int64  
 8   imdb_id     1100 non-null   object 
 9   poster      1036 non-null   object 
 10  top250      1100 non-null   int64  
 11  top250tv    1100 non-null   int64  
 12  title_date  1100 non-null   object 
dtypes: float64(1), int64(5), object(7)
memory usage: 111.8+ KB


In [5]:
#display first 4 rows
display(dfmcn.head())

,title,img,type,netflix_id,synopsis,rating,year,runtime,imdb_id,poster,top250,top250tv,title_date
0,The Goat and Her Three Kids,https://occ-0-2758-3467.1.nflxso.net/dnm/api/v6/WNk1mr9x_Cd_2itp6pUM7-lXMJg/AAAABcAhg7Ed9mUM0OwBbe0DMWKmi9W4HcbSYhpju4xeMRr1WODB1iVTACz8_XS7JP32HKU5UmHoy0a-5hh70yVj9eYehl1jSatX3g.jpg?r=852,movie,81677719,A determined widow must protect her three children from a bloodthirsty neighbor bent on revenge in this gruesome spin on the beloved fairy tale.,9.8,2022,4972,tt9243412,https://m.media-amazon.com/images/M/MV5BNWUxOGM3ZWUtOTUzZS00ODQwLTlkMmUtZjhkYTNhM2JkNGUxXkEyXkFqcGdeQXVyNTI0MjQyMDU@._V1_SX300.jpg,0,0,24/3/23
1,Luciano Mellera: Infantiloide,http://occ-0-2851-38.1.nflxso.net/dnm/api/v6/evlCitJPPCVCry0BZlEFb5-QjKc/AAAABZPbNynCLoKGa8apkY2aS3R81-qI5FOlRL1OqxFHKPsDJzrJIHtr6l2ZhRPVG7TmfDuL2ShqjH7tv9kApsbWAOIkskTbuAUa5SswK8ZkDBx_27E9UTZHMaUM2Mw.jpg?r=2f1,movie,80192064,ArgentinaÔs Luciano Mellera emphasizes the humorous and fantastical aspects of childhood through comedic impersonations and insights on daily life.,9.6,2018,3962,tt7607400,NaN,0,0,7/7/18
2,Ether,https://occ-0-3952-3467.1.nflxso.net/dnm/api/v6/evlCitJPPCVCry0BZlEFb5-QjKc/AAAABb-w8bi8xCn_2zl1rLbsrjw8c8_gWO7pZdzyPvqjm0PWgAs1Tj_NSMpS6BNh4eB7F7hFEqqQY9x8byP7TpDkuu-vEw.jpg?r=2c6,movie,81565048,"After a stint in prison, an unscrupulous doctor obsessed with science becomes a military medic and performs dangerous experiments for sinister reasons.",9.6,2019,7047,tt2570120,https://m.media-amazon.com/images/M/MV5BMjIwODQwMzk0M15BMl5BanBnXkFtZTcwMzcyODkwOQ@@._V1_SX300.jpg,0,0,1/3/22
3,Youth v Gov,https://occ-0-2851-1432.1.nflxso.net/dnm/api/v6/evlCitJPPCVCry0BZlEFb5-QjKc/AAAABUA3DBkSa6TxiVsxUrRw76pNHDTFbgvdA8QY2J6p_uHK0NAxghaNwRtauqkVba8HNh0iyggKlFkJvETqFi-FxmkSKQ.jpg?r=5bd,movie,81586492,This documentary follows the journey of 21 young plaintiffs suing the US government for its actions in creating the climate crisis.,9.6,2020,6554,tt13192640,https://m.media-amazon.com/images/M/MV5BM2ZjMzAxZDgtOWUxZi00Y2ZlLThjMTUtOTM0MjRkM2YxMzk4XkEyXkFqcGdeQXVyMTA3NzE5NDY4._V1_SX300.jpg,0,0,29/4/22
4,Stars in the Sky: A Hunting Story,https://occ-0-2851-1432.1.nflxso.net/dnm/api/v6/evlCitJPPCVCry0BZlEFb5-QjKc/AAAABYXo3vPKH6WlxCONV--TUgdwsk6KuMrWe3f2nFYIg6GggvURtsuwbDE3bNW5611lIyDMjJFqBoXOaTAZGYcQJ7yZGw.jpg?r=708,movie,81292225,"This documentary follows a group of hunters as they grapple with the complexities, controversies, and contradictions of pursuing animals in the wild.",9.5,2018,4151,tt10078502,https://m.media-amazon.com/images/M/MV5BMGFlZjhhOGMtNGY1ZS00YzViLWJjYzktNGQ2MDA4OTE2OGJmXkEyXkFqcGdeQXVyOTYwMjIzNTQ@._V1_SX300.jpg,0,0,7/8/20


In [6]:
#Changing the title date to date format
dfmcn["title_date"] = pd.to_datetime(dfmcn["title_date"])

In [7]:
#Removing the features which contains the url and one unique value
dfmcn.drop(columns=['img','poster','type'],inplace=True)
dfmcn.reset_index(drop=True,inplace=True)

In [8]:
dfmcn.head()

,title,netflix_id,synopsis,rating,year,runtime,imdb_id,top250,top250tv,title_date
0,The Goat and Her Three Kids,81677719,A determined widow must protect her three children from a bloodthirsty neighbor bent on revenge in this gruesome spin on the beloved fairy tale.,9.8,2022,4972,tt9243412,0,0,2023-03-24
1,Luciano Mellera: Infantiloide,80192064,ArgentinaÔs Luciano Mellera emphasizes the humorous and fantastical aspects of childhood through comedic impersonations and insights on daily life.,9.6,2018,3962,tt7607400,0,0,2018-07-07
2,Ether,81565048,"After a stint in prison, an unscrupulous doctor obsessed with science becomes a military medic and performs dangerous experiments for sinister reasons.",9.6,2019,7047,tt2570120,0,0,2022-01-03
3,Youth v Gov,81586492,This documentary follows the journey of 21 young plaintiffs suing the US government for its actions in creating the climate crisis.,9.6,2020,6554,tt13192640,0,0,2022-04-29
4,Stars in the Sky: A Hunting Story,81292225,"This documentary follows a group of hunters as they grapple with the complexities, controversies, and contradictions of pursuing animals in the wild.",9.5,2018,4151,tt10078502,0,0,2020-07-08


In [9]:
#Finding the unique values
dfmcn.nunique()

title         1084
netflix_id    1100
synopsis      1094
rating          29
year             6
runtime        984
imdb_id       1062
top250          11
top250tv         2
title_date     683
dtype: int64

In [10]:
#Finding which synopsis has more than one sample
dfmcn.groupby('synopsis')['netflix_id'].nunique().sort_values(ascending=False)

synopsis
A brash but brilliant Indian intelligence agent leads a covert operation to nab the mastermind behind a series of attacks threatening national security.            4
When a private school teacher is assigned to a neglected public school, he must overcome personal and political strife in his fight for education.                  2
An aspiring musician battles age-old caste divides to be able to learn the art of a classical instrument from a traditionalist, veteran percussionist.              2
An affable, newly appointed college warden proves to be no ordinary man when an old enemy resurfaces and exposes his complicated, violent past.                     2
"Queen of Sleaze" or feminist pioneer? Powerhouse novelist Jackie Collins created a brash sex-positive persona while hiding her private struggles.                  1
Over the course of 30 years, two couples experience tragedy, heartbreak and redemption as seismic shifts in Chinese society transform their world.               

In [11]:
dfmcn[dfmcn['synopsis']=='A brash but brilliant Indian intelligence agent leads a covert operation to nab the mastermind behind a series of attacks threatening national security.']

,title,netflix_id,synopsis,rating,year,runtime,imdb_id,top250,top250tv,title_date
360,Wild Dog,81427455,A brash but brilliant Indian intelligence agent leads a covert operation to nab the mastermind behind a series of attacks threatening national security.,7.7,2020,7574,tt11503178,0,0,2021-04-22
361,Wild Dog,81444683,A brash but brilliant Indian intelligence agent leads a covert operation to nab the mastermind behind a series of attacks threatening national security.,7.7,2020,7574,tt11503178,0,0,2021-04-22
362,Wild Dog,81444684,A brash but brilliant Indian intelligence agent leads a covert operation to nab the mastermind behind a series of attacks threatening national security.,7.7,2020,7574,tt11503178,0,0,2021-04-22
363,Wild Dog,81444685,A brash but brilliant Indian intelligence agent leads a covert operation to nab the mastermind behind a series of attacks threatening national security.,7.7,2020,7574,tt11503178,0,0,2021-04-22


In [12]:
dfmcn[dfmcn['synopsis']=='When a private school teacher is assigned to a neglected public school, he must overcome personal and political strife in his fight for education.']

,title,netflix_id,synopsis,rating,year,runtime,imdb_id,top250,top250tv,title_date
281,Vaathi,81671561,"When a private school teacher is assigned to a neglected public school, he must overcome personal and political strife in his fight for education.",7.9,2023,8052,tt15427980,0,0,2023-03-17
401,Sir,81681253,"When a private school teacher is assigned to a neglected public school, he must overcome personal and political strife in his fight for education.",7.7,2023,8059,tt7142506,0,0,2023-03-18


In [13]:
dfmcn[dfmcn['synopsis']=='An aspiring musician battles age-old caste divides to be able to learn the art of a classical instrument from a traditionalist, veteran percussionist.']

,title,netflix_id,synopsis,rating,year,runtime,imdb_id,top250,top250tv,title_date
30,Sarvam Thaala Mayam (Tamil Version),81083971,"An aspiring musician battles age-old caste divides to be able to learn the art of a classical instrument from a traditionalist, veteran percussionist.",8.8,2018,7841,tt7719976,0,0,2019-01-03
31,Sarvam Thaala Mayam (Telugu Version),81074135,"An aspiring musician battles age-old caste divides to be able to learn the art of a classical instrument from a traditionalist, veteran percussionist.",8.8,2018,7841,tt7719976,0,0,2019-08-04


<b> Checking Null values in all Columns </b>

In [14]:
dfmcn.isnull().sum()

title         0
netflix_id    0
synopsis      0
rating        0
year          0
runtime       0
imdb_id       0
top250        0
top250tv      0
title_date    0
dtype: int64

In [15]:
#Changing to lower case
dfmcn["text_clean"] = dfmcn["synopsis"].apply(lambda x: x.lower())
display(dfmcn.head())

,title,netflix_id,synopsis,rating,year,runtime,imdb_id,top250,top250tv,title_date,text_clean
0,The Goat and Her Three Kids,81677719,A determined widow must protect her three children from a bloodthirsty neighbor bent on revenge in this gruesome spin on the beloved fairy tale.,9.8,2022,4972,tt9243412,0,0,2023-03-24,a determined widow must protect her three children from a bloodthirsty neighbor bent on revenge in this gruesome spin on the beloved fairy tale.
1,Luciano Mellera: Infantiloide,80192064,ArgentinaÔs Luciano Mellera emphasizes the humorous and fantastical aspects of childhood through comedic impersonations and insights on daily life.,9.6,2018,3962,tt7607400,0,0,2018-07-07,argentinaôs luciano mellera emphasizes the humorous and fantastical aspects of childhood through comedic impersonations and insights on daily life.
2,Ether,81565048,"After a stint in prison, an unscrupulous doctor obsessed with science becomes a military medic and performs dangerous experiments for sinister reasons.",9.6,2019,7047,tt2570120,0,0,2022-01-03,"after a stint in prison, an unscrupulous doctor obsessed with science becomes a military medic and performs dangerous experiments for sinister reasons."
3,Youth v Gov,81586492,This documentary follows the journey of 21 young plaintiffs suing the US government for its actions in creating the climate crisis.,9.6,2020,6554,tt13192640,0,0,2022-04-29,this documentary follows the journey of 21 young plaintiffs suing the us government for its actions in creating the climate crisis.
4,Stars in the Sky: A Hunting Story,81292225,"This documentary follows a group of hunters as they grapple with the complexities, controversies, and contradictions of pursuing animals in the wild.",9.5,2018,4151,tt10078502,0,0,2020-07-08,"this documentary follows a group of hunters as they grapple with the complexities, controversies, and contradictions of pursuing animals in the wild."


In [16]:
#Removing stop words
# Download NLTK stopwords (if not already downloaded)
nltk.download('stopwords')
from nltk.corpus import stopwords
# Define a function to remove stopwords
def remove_stopwords(text):
    stop_words = set(stopwords.words('english'))
    words = nltk.word_tokenize(text)
    filtered_words = [word for word in words if word.lower() not in stop_words]
    return ' '.join(filtered_words)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Dell\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [17]:
# Apply the remove_stopwords function to the 'Text' column
dfmcn['text_clean'] = dfmcn['text_clean'].apply(remove_stopwords)
display(dfmcn.head())

,title,netflix_id,synopsis,rating,year,runtime,imdb_id,top250,top250tv,title_date,text_clean
0,The Goat and Her Three Kids,81677719,A determined widow must protect her three children from a bloodthirsty neighbor bent on revenge in this gruesome spin on the beloved fairy tale.,9.8,2022,4972,tt9243412,0,0,2023-03-24,determined widow must protect three children bloodthirsty neighbor bent revenge gruesome spin beloved fairy tale .
1,Luciano Mellera: Infantiloide,80192064,ArgentinaÔs Luciano Mellera emphasizes the humorous and fantastical aspects of childhood through comedic impersonations and insights on daily life.,9.6,2018,3962,tt7607400,0,0,2018-07-07,argentinaôs luciano mellera emphasizes humorous fantastical aspects childhood comedic impersonations insights daily life .
2,Ether,81565048,"After a stint in prison, an unscrupulous doctor obsessed with science becomes a military medic and performs dangerous experiments for sinister reasons.",9.6,2019,7047,tt2570120,0,0,2022-01-03,"stint prison , unscrupulous doctor obsessed science becomes military medic performs dangerous experiments sinister reasons ."
3,Youth v Gov,81586492,This documentary follows the journey of 21 young plaintiffs suing the US government for its actions in creating the climate crisis.,9.6,2020,6554,tt13192640,0,0,2022-04-29,documentary follows journey 21 young plaintiffs suing us government actions creating climate crisis .
4,Stars in the Sky: A Hunting Story,81292225,"This documentary follows a group of hunters as they grapple with the complexities, controversies, and contradictions of pursuing animals in the wild.",9.5,2018,4151,tt10078502,0,0,2020-07-08,"documentary follows group hunters grapple complexities , controversies , contradictions pursuing animals wild ."


In [18]:
#Now, different variations of same word typically exist in a corpus. For example, "go", "going" are variations of same root word.
#Stemming refers to removal of suffices, such as "ing", "ly". 
#Lemmatization on the other hand converts the word to its root word, rather than just removing the suffices.
#Lammatizing the text
nltk.download('wordnet')
nltk.download('omw-1.4')
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

def lemmatize_text(text):
    words = [lemmatizer.lemmatize(word) for word in text.split()]
    text = ' '.join(words)
    return text

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Dell\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\Dell\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [19]:
dfmcn["text_clean"] = dfmcn["text_clean"].apply(lemmatize_text)
dfmcn["text_clean"].sample(5, random_state=42)

328             dedicated government official put career jeopardy insisting india become nuclear power , goal powerful force resist .
688     dazzling tender-hearted , legendary astrologer walter mercado vanished peak fame . documentary poignantly explains happened .
413                     16-year-old daughter go missing , david trace last digital step using laptop , social medium detective find .
788    buoyed formidable wife lynne , dick cheney gain power shrewdly manipulates u.s. vice presidency explosive global consequence .
244      charles townsendôs agency expands abroad , two established angel team new recruit fend enemy securing revolutionary device .
Name: text_clean, dtype: object

In [20]:
display(dfmcn.head())

,title,netflix_id,synopsis,rating,year,runtime,imdb_id,top250,top250tv,title_date,text_clean
0,The Goat and Her Three Kids,81677719,A determined widow must protect her three children from a bloodthirsty neighbor bent on revenge in this gruesome spin on the beloved fairy tale.,9.8,2022,4972,tt9243412,0,0,2023-03-24,determined widow must protect three child bloodthirsty neighbor bent revenge gruesome spin beloved fairy tale .
1,Luciano Mellera: Infantiloide,80192064,ArgentinaÔs Luciano Mellera emphasizes the humorous and fantastical aspects of childhood through comedic impersonations and insights on daily life.,9.6,2018,3962,tt7607400,0,0,2018-07-07,argentinaôs luciano mellera emphasizes humorous fantastical aspect childhood comedic impersonation insight daily life .
2,Ether,81565048,"After a stint in prison, an unscrupulous doctor obsessed with science becomes a military medic and performs dangerous experiments for sinister reasons.",9.6,2019,7047,tt2570120,0,0,2022-01-03,"stint prison , unscrupulous doctor obsessed science becomes military medic performs dangerous experiment sinister reason ."
3,Youth v Gov,81586492,This documentary follows the journey of 21 young plaintiffs suing the US government for its actions in creating the climate crisis.,9.6,2020,6554,tt13192640,0,0,2022-04-29,documentary follows journey 21 young plaintiff suing u government action creating climate crisis .
4,Stars in the Sky: A Hunting Story,81292225,"This documentary follows a group of hunters as they grapple with the complexities, controversies, and contradictions of pursuing animals in the wild.",9.5,2018,4151,tt10078502,0,0,2020-07-08,"documentary follows group hunter grapple complexity , controversy , contradiction pursuing animal wild ."


In [21]:
dfmcn.shape

(1100, 11)

## TF-IDF Matrix

The TF-IDF(term frequency-inverse document frequency) is used to evaluate how relevant a word is to a document. TF-IDF for a word is calculated by multiplying two different metrics:
+ Term frequency: Frequency of occurence of a word in a document
+ Inverse document frequency: Calculating the occurence of a word across documents. If the value is close to 0, the more common the word is, i.e. occurs in many documents.

The higher the TF-IDF score, the more relevant the word is in the particular document. 

In [22]:
# TF-IDF vector representation
vectorizer = TfidfVectorizer(stop_words='english',
                     ngram_range = (1,3))
doc_vectors = vectorizer.fit_transform(dfmcn['text_clean'])
print(doc_vectors.shape)
print(vectorizer.get_feature_names_out())

(1100, 31284)
['000' '000 round' '000 round competition' ... 'ôs' 'ôs celebrated'
 'ôs celebrated 2018']


In [23]:
# type(doc_vectors)

In [24]:
# doc_vectors.toarray()

In [25]:
# # save the TfidfVectorizer to disk
# filename_TFidfVec = 'Tfidf_vec_cosine_phoneyes.sav'
# pickle.dump(vectorizer, open(filename_TFidfVec, 'wb'))

In [26]:
# result=[]
# query="A brash but brilliant Indian intelligence agent leads a covert operation to nab the mastermind behind a series of attacks threatening national security."
# query = query.lower()
# query = remove_stopwords(query)
# query = lemmatize_text(query)
# q_vector = vectorizer.transform([query])
# print("Comparable Description: ", query)
# result=cosine_similarity(q_vector, doc_vectors.toarray())
# print(result)

In [27]:
# elem_list2=[]
# for i in result[:10]:
#     for elem in i:
#         #print("Review",f, "Similarity: ", elem)
#         elem_list2.append(elem)
# type(elem_list2)

In [28]:
# print(len(elem_list2))
# print(max(elem_list2))

In [29]:
dfmcn.columns

Index(['title', 'netflix_id', 'synopsis', 'rating', 'year', 'runtime',
       'imdb_id', 'top250', 'top250tv', 'title_date', 'text_clean'],
      dtype='object')

In [30]:
col_listB=['title','rating','year','synopsis','netflix_id']

In [31]:
def comp_description2(query):
        results=[]
        query = query.lower()
        query = remove_stopwords(query)
        query = lemmatize_text(query)
        q_vector = vectorizer.transform([query])
        print("Comparable Description: ", query)
        results.append(cosine_similarity(q_vector, doc_vectors.toarray()))
        elem_list=[]
#         print("result",results)
        for i in results[:10]:
            for elem in i[0]:
                elem_list.append(elem)
        
#         print("type of elem",type(elem_list))
        sorted_listA = sorted(elem_list, reverse=True)

        if (sum(elem_list)/len(elem_list)).any() == 0.0:
            print("\n No similar descriptions")
        else:
            print("\n The Description Most similar to the Comparable Description is Description wit Index #" ,elem_list.index(max(sorted_listA)))
            print("\n Max Similarity score: ", max(sorted_listA),"\n")
            

            indexes_to_display=[]

            unique_scores = list(set(elem_list))  # Get unique values from the list
            unique_scores.sort(reverse=True)     # Sort in descending order
            first_max_value = unique_scores[0] 
            second_max_value = unique_scores[1] 
            third_max_value = unique_scores[2] 
            first_max_indices = [index for index, value in enumerate(elem_list) if value == first_max_value]
            second_max_indices = [index for index, value in enumerate(elem_list) if value == second_max_value]
            third_max_indices = [index for index, value in enumerate(elem_list) if value == third_max_value]
            indexes_to_display = first_max_indices +  second_max_indices + third_max_indices

            # Display data of the specific indexes using iloc
            selected_rows = dfmcn[col_listB].iloc[indexes_to_display]
            
#           print(selected_rows)
            display(selected_rows)

<b> Knowledge Assit </b>

In [32]:
comp_description2("When a private school teacher is assigned to a neglected public school, he must overcome personal and political strife in his fight for education.")

Comparable Description:  private school teacher assigned neglected public school , must overcome personal political strife fight education .

 The Description Most similar to the Comparable Description is Description wit Index # 281

 Max Similarity score:  1.0000000000000009 



,title,rating,year,synopsis,netflix_id
281,Vaathi,7.9,2023,"When a private school teacher is assigned to a neglected public school, he must overcome personal and political strife in his fight for education.",81671561
401,Sir,7.7,2023,"When a private school teacher is assigned to a neglected public school, he must overcome personal and political strife in his fight for education.",81681253
183,Wild Little Love,8.1,2019,"Forced by her dad to attend public school, a popular private school prankster tries to survive her new environment and embarks on an unlikely romance.",81437351
986,Possessed,7.1,2022,"A teacher must put aside his personal traumas to rally his school in a fight for survival against a group of violent, possessed students.",81631032
